In [ ]:
from geneformer import InSilicoPerturber
from geneformer import InSilicoPerturberStats
from geneformer import EmbExtractor

### in silico perturbation in deletion mode to determine genes whose deletion in the dilated cardiomyopathy (dcm) state significantly shifts the embedding towards non-failing (nf) state

In [ ]:
# first obtain start, goal, and alt embedding positions
# this function was changed to be separate from perturb_data
# to avoid repeating calcuations when parallelizing perturb_data
cell_states_to_model={"state_key": "disease", 
                      "start_state": "dcm", 
                      "goal_state": "nf", 
                      "alt_states": ["hcm"]}

filter_data_dict={"cell_type":["Cardiomyocyte1","Cardiomyocyte2","Cardiomyocyte3"]}

# OF NOTE: model_version should match version of model to be used (V1 or V2) to use the correct token dictionary
embex = EmbExtractor(model_type="CellClassifier", # if using previously fine-tuned cell classifier model
                     num_classes=3,
                     filter_data=filter_data_dict,
                     max_ncells=1000,
                     emb_layer=0,
                     summary_stat="exact_mean",
                     forward_batch_size=256,
                     model_version="V1",  # OF NOTE: SET TO V1 MODEL, PROVIDE V1 MODEL PATH IN SUBSEQUENT CODE
                     nproc=16)

state_embs_dict = embex.get_state_embs(cell_states_to_model,
                                       "../fine_tuned_models/gf-6L-30M-i2048_CellClassifier_cardiomyopathies_220224", # example 30M fine-tuned model
                                       "path/to/input_data",
                                       "path/to/output_directory",
                                       "output_prefix")

In [ ]:
# OF NOTE: model_version should match version of model to be used (V1 or V2) to use the correct token dictionary
isp = InSilicoPerturber(perturb_type="delete",
                        perturb_rank_shift=None,
                        genes_to_perturb="all",
                        combos=0,
                        anchor_gene=None,
                        model_type="CellClassifier", # if using previously fine-tuned cell classifier model
                        num_classes=3,
                        emb_mode="cell",  # OF NOTE: SET TO "CELL" FOR V1 MODEL. FOR V2, SHOULD BE "CLS" (current default).
                        cell_emb_style="mean_pool",
                        filter_data=filter_data_dict,
                        cell_states_to_model=cell_states_to_model,
                        state_embs_dict=state_embs_dict,
                        max_ncells=2000,
                        emb_layer=0,
                        forward_batch_size=400,
                        model_version="V1",  # OF NOTE: SET TO V1 MODEL, PROVIDE V1 MODEL PATH IN SUBSEQUENT CODE
                        nproc=16)

In [ ]:
# outputs intermediate files from in silico perturbation

isp.perturb_data("../fine_tuned_models/Geneformer-V1-10M_CellClassifier_cardiomyopathies_220224", # example V1 fine-tuned model
                 "path/to/input_data",
                 "path/to/isp_output_directory",
                 "output_prefix")

In [ ]:
# OF NOTE: model_version should match version of model to be used (V1 or V2) to use the correct token dictionary
ispstats = InSilicoPerturberStats(mode="goal_state_shift",
                                  genes_perturbed="all",
                                  combos=0,
                                  anchor_gene=None,
                                  cell_states_to_model=cell_states_to_model,
                                  model_version="V1",  # OF NOTE: SET TO V1 MODEL SINCE V1 WAS USED FOR IN SILICO PERTURBATION ABOVE)

In [ ]:
# extracts data from intermediate files and processes stats to output in final .csv
ispstats.get_stats("path/to/isp_output_directory", # this should be the directory 
                   None,
                   "path/to/isp_stats_output_directory",
                   "output_prefix")